# Optiver Realized Volatility Prediction

## Apply your data science skills to make financial markets better


<a href= "https://www.optiver.com/insights/guides/options-volatility/">Volatility</a> is one of the most prominent terms you’ll hear on any trading floor – and for good reason. In financial markets, volatility captures the amount of fluctuation in prices. High volatility is associated to periods of market turbulence and to large price swings, while low volatility describes more calm and quiet markets. For trading firms like Optiver, accurately predicting volatility is essential for the trading of options, whose price is <a href= "https://www.optiver.com/insights/guides/options-pricing/">directly related to the volatility</a> of the underlying product.

## Understanding the data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn

train.csv The ground truth values for the training set.

* stock_id - Same as above, but since this is a csv the column will load as an integer instead of categorical.

* time_id - Same as above.

* target - The realized volatility computed over the 10 minute window following the feature data under the same stock/time_id. There is no overlap between feature and target data. You can find more info in this <a href= "https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data?scriptVersionId=67183666#Competition-data">tutorial notebook</a>.

In [2]:
train = pd.read_csv("drive/MyDrive/optiver-realized-volatility-prediction/train.csv")
print(train.info())
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428932 entries, 0 to 428931
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   stock_id  428932 non-null  int64  
 1   time_id   428932 non-null  int64  
 2   target    428932 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 9.8 MB
None


,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [ ]:
id_count = pd.DataFrame(dftt.stock_id.value_counts()).sort_index()
print(id_count)

     stock_id
0        3830
1        3830
2        3830
3        3830
4        3830
..        ...
122      3830
123      3830
124      3830
125      3830
126      3830

[112 rows x 1 columns]


In [ ]:
dftt.tail()

,stock_id,time_id,target
428927,126,32751,0.003461
428928,126,32753,0.003113
428929,126,32758,0.004070
428930,126,32763,0.003357
428931,126,32767,0.002090


test.csv Provides the mapping between the other data files and the submission file. As with other test files, most of the data is only available to your notebook upon submission with just the first few rows available for download.


* stock_id - Same as above.

* time_id - Same as above.

* row_id - Unique identifier for the submission row. There is one row for each existing time ID/stock ID pair. Each time window is not necessarily containing every individual stock.

In [ ]:
dfts = pd.read_csv("test.csv")
print(dfts.info())
dfts.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   stock_id  3 non-null      int64 
 1   time_id   3 non-null      int64 
 2   row_id    3 non-null      object
dtypes: int64(2), object(1)
memory usage: 200.0+ bytes
None


,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


sample_submission.csv - A sample submission file in the correct format.

* row_id - Same as in test.csv.

* target - Same definition as in train.csv. The benchmark is using the median target value from train.csv.

In [ ]:
dfs = pd.read_csv("sample_submission.csv")
print(dfs.info())
dfs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   row_id  3 non-null      object 
 1   target  3 non-null      float64
dtypes: float64(1), object(1)
memory usage: 176.0+ bytes
None


,row_id,target
0,0-4,0.003048
1,0-32,0.003048
2,0-34,0.003048


## Selecting data

In [3]:
import os
from sklearn.metrics import r2_score
import glob
list_order_book_file_train = glob.glob('drive/MyDrive/optiver-realized-volatility-prediction/book_train.parquet/*')

### Log returns

Returns are widely used in finance, however log returns are preferred whenever some mathematical modelling is required. Calling  St  the price of the stock  S  at time  t , we can define the log return between  t1  and  t2  as:

rt1,t2=log(St2St1)

In [4]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

### Realized volatility

When we trade options, a valuable input to our models is the standard deviation of the stock log returns. The standard deviation will be different for log returns computed over longer or shorter intervals, for this reason it is usually normalized to a 1-year period and the annualized standard deviation is called volatility.

σ=(∑t (rt-1,t)↑2)↑(1/2)

In [5]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
#realized_vol = realized_volatility(book_example['log_return'])
#print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

In [6]:
def realized_volatility_per_time_id(file_path, prediction_column_name):
    
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [7]:
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    
    return df_past_realized

df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                           prediction_column_name='pred')

In [14]:
# for a single file
def realized_volatility_per_time_id_1(file_path, prediction_column_name):
    
    #df_book_data = pd.read_parquet(file_path)
    df_book_data = pd.read_csv(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

def past_realized_volatility_per_stock_1(file_1,prediction_column_name):
    
    df_past_realized = pd.DataFrame()
    #for file in list_file:
    df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id_1(file_1,prediction_column_name)])
    return df_past_realized

In [8]:
df_past_realized_train.head()

,row_id,pred
0,99-5,0.002085
1,99-11,0.001024
2,99-16,0.001888
3,99-31,0.001481
4,99-62,0.000994


In [9]:
# joining the output dataframe with train.csv to see the performance of the naive prediction on training set.

train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')


In [10]:
df_joined.head()

,row_id,target,pred
0,0-5,0.004136,0.004499
1,0-11,0.001445,0.001204
2,0-16,0.002168,0.002369
3,0-31,0.002195,0.002574
4,0-62,0.001747,0.001894


In [11]:
# Evaluating the naive prediction result by two metrics: RMSPE and R squared.

from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')

Performance of the naive prediction: R2 score: 0.628, RMSPE: 0.341


## Submission

In [12]:
list_order_book_file_test = glob.glob('drive/MyDrive/optiver-realized-volatility-prediction/book_test.parquet/*')
df_naive_pred_test = past_realized_volatility_per_stock(list_file=list_order_book_file_test,
                                                           prediction_column_name='target')
df_naive_pred_test.to_csv('submission.csv',index = False)

In [13]:
df_naive_pred_test.head()


,row_id,target
0,0-4,0.000294


In [15]:
list_order_book_file_test = 'drive/MyDrive/optiver-realized-volatility-prediction/test.csv'
df_naive_pred_test = past_realized_volatility_per_stock_1(file_1=list_order_book_file_test,
                                                           prediction_column_name='target')
df_naive_pred_test.to_csv('submission.csv',index = False)

KeyError: ignored